<pre>Inspired by Anton Enns's Kernel
https://www.kaggle.com/tocha4/lanl-master-s-approach</pre>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
from tqdm import tqdm_notebook
import datetime
import time
import random
from joblib import Parallel, delayed

from catboost import Pool, CatBoostRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV, SelectFromModel

In [2]:
# This is NN Model creation
from keras.models import *
from keras.optimizers import *
from keras.regularizers import *
from keras.layers import * # Keras is the most friendly Neural Network library, this Kernel use a lot of layers classes
from keras import backend as K # The backend give us access to tensorflow operations and allow us to create the Attention class
from keras.callbacks import *

Using TensorFlow backend.


In [3]:
print(os.listdir('../input/'))
train_X_0 = pd.read_csv("../input/lanl-master-s-features-creating-0/train_X_features_865.csv")
train_X_1 = pd.read_csv("../input/lanl-master-s-features-creating-1/train_X_features_865.csv")
y_0 = pd.read_csv("../input/lanl-master-s-features-creating-0/train_y.csv", index_col=False,  header=None)
y_1 = pd.read_csv("../input/lanl-master-s-features-creating-1/train_y.csv", index_col=False,  header=None)

['andrews-new-script-plus-a-genetic-program-model', 'lanl-master-s-features-creating-1', 'LANL-Earthquake-Prediction', 'lanl-master-s-features-creating-0']


In [4]:
train_X = pd.concat([train_X_0, train_X_1], axis=0)
train_X = train_X.reset_index(drop=True)
print(train_X.shape)
train_X.head()

(33000, 865)


,FFT_Mag_01q0,FFT_Mag_10q0,FFT_Mag_90q0,FFT_Mag_99q0,FFT_Mag_mean0,FFT_Mag_std0,FFT_Mag_max0,FFT_Phz_mean0,FFT_Phz_std0,FFT_Mag_01q2500,FFT_Mag_10q2500,FFT_Mag_90q2500,FFT_Mag_99q2500,FFT_Mag_mean2500,FFT_Mag_std2500,FFT_Mag_max2500,FFT_Phz_mean2500,FFT_Phz_std2500,FFT_Mag_01q5000,FFT_Mag_10q5000,FFT_Mag_90q5000,FFT_Mag_99q5000,FFT_Mag_mean5000,FFT_Mag_std5000,FFT_Mag_max5000,FFT_Phz_mean5000,FFT_Phz_std5000,FFT_Mag_01q7500,FFT_Mag_10q7500,FFT_Mag_90q7500,FFT_Mag_99q7500,FFT_Mag_mean7500,FFT_Mag_std7500,FFT_Mag_max7500,FFT_Phz_mean7500,FFT_Phz_std7500,FFT_Mag_01q10000,FFT_Mag_10q10000,FFT_Mag_90q10000,FFT_Mag_99q10000,...,q01_roll_std_100,q05_roll_std_100,q95_roll_std_100,q99_roll_std_100,av_change_abs_roll_std_100,av_change_rate_roll_std_100,abs_max_roll_std_100,ave_roll_mean_100,std_roll_mean_100,max_roll_mean_100,min_roll_mean_100,q01_roll_mean_100,q05_roll_mean_100,q95_roll_mean_100,q99_roll_mean_100,av_change_abs_roll_mean_100,av_change_rate_roll_mean_100,abs_max_roll_mean_100,ave_roll_std_1000,std_roll_std_1000,max_roll_std_1000,min_roll_std_1000,q01_roll_std_1000,q05_roll_std_1000,q95_roll_std_1000,q99_roll_std_1000,av_change_abs_roll_std_1000,av_change_rate_roll_std_1000,abs_max_roll_std_1000,ave_roll_mean_1000,std_roll_mean_1000,max_roll_mean_1000,min_roll_mean_1000,q01_roll_mean_1000,q05_roll_mean_1000,q95_roll_mean_1000,q99_roll_mean_1000,av_change_abs_roll_mean_1000,av_change_rate_roll_mean_1000,abs_max_roll_mean_1000
0,139.890381,430.720962,3841.665607,7039.062475,1747.987667,1505.807396,12964.190508,-0.019064,0.917755,580.989637,1788.141759,10385.447859,17303.410477,5576.975550,3670.078510,21851.886653,-0.018764,0.904588,576.378990,2244.232557,10573.085626,15464.766090,6118.346896,3304.632470,24716.397978,0.021492,0.912655,698.174578,2463.552257,13396.596838,21973.239590,7353.612733,4529.556526,32176.902122,-0.006021,0.909976,304.096958,992.766285,8185.997878,13830.325240,...,2.341954,2.528125,12.600701,29.647864,-4.876389e-05,74991.820387,64.916157,4.429363,0.546202,13.91,-6.31,3.12,3.73,5.12,5.66,0.000002,74986.645144,13.91,5.261040,4.761211,37.834871,2.542905,2.642355,2.753961,14.856489,31.176557,-1.579223e-05,74576.876580,37.834871,4.429468,0.237930,5.415,3.327,3.882,4.029,4.804,4.941,-7.516779e-07,74577.679964,5.415
1,140.598496,431.872538,3093.828306,5914.712982,1545.320042,1239.738638,14530.791553,0.002910,0.904066,371.779862,1247.165814,8126.085534,13429.208174,4307.232346,2810.568152,16980.104876,-0.000715,0.909822,758.474860,2403.552302,10859.395754,15352.256883,6500.555291,3294.129910,19523.499828,-0.012594,0.903179,549.766537,1893.113362,11412.308979,18897.650880,6049.687590,3897.710996,26050.265072,0.006173,0.900904,232.076227,898.383220,5984.967037,11639.375545,...,2.320288,2.486407,9.477645,22.710339,3.857706e-07,74941.498345,87.311820,4.241309,0.531692,20.71,-5.29,3.08,3.54,4.94,5.32,-0.000003,74937.945900,20.71,4.673184,4.222990,40.883127,2.512763,2.624581,2.734313,11.228717,24.813733,3.154961e-07,74486.700394,40.883127,4.241174,0.262043,5.362,3.139,3.636,3.792,4.681,4.865,-2.791946e-06,74484.368615,5.362
2,123.395925,364.114775,2052.051388,3594.307716,1157.415026,765.798779,11148.510210,-0.029014,0.904867,326.587430,974.239681,4765.050989,7051.493436,2682.743587,1499.339342,9153.525311,-0.023570,0.915614,391.627693,1548.206760,7269.258931,10307.151670,4170.463168,2252.933240,15298.295361,-0.018223,0.891113,581.302261,1843.330646,11490.306704,18905.302322,6056.587080,3985.532867,28425.965355,0.008742,0.898519,297.355997,852.686652,5616.317474,10041.900327,...,2.330388,2.500323,10.445704,17.818192,9.870522e-07,75071.732023,34.908566,4.616689,0.425430,8.29,1.64,3.61,3.93,5.29,5.65,0.000004,75068.641763,8.29,4.591089,2.652471,17.429133,2.524292,2.674261,2.731556,10.600250,15.267357,5.168715e-06,74654.822041,17.429133,4.615960,0.239739,5.312,3.932,4.029,4.180,5.004,5.102,2.221477e-06,74653.840860,5.312
3,92.046507,370.215407,2011.169915,3849.050613,1125.200572,817.604731,19980.904036,-0.002131,0.9052

In [5]:
y = pd.concat([y_0, y_1], axis=0)
y = y.reset_index(drop=True)
y[0].shape

(33000,)

In [6]:
train_y = pd.Series(y[0].values)

In [7]:
test_X = pd.read_csv("../input/lanl-master-s-features-creating-0/test_X_features_10.csv")
# del X["seg_id"], test_X["seg_id"]

In [8]:
scaler = StandardScaler()
train_columns = train_X.columns

train_X[train_columns] = scaler.fit_transform(train_X[train_columns])
test_X[train_columns] = scaler.transform(test_X[train_columns])

<pre>CAT BOOST Algorithm</pre>

In [9]:
train_columns = train_X.columns
n_fold = 5

In [10]:
%%time
folds = KFold(n_splits=n_fold, shuffle = True, random_state=42)

oof = np.zeros(len(train_X))
train_score = []
fold_idxs = []
# if PREDICTION: 
predictions = np.zeros(len(test_X))

feature_importance_df = pd.DataFrame()
#run model
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X,train_y.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    fold_idxs.append(val_idx)

    X_tr, X_val = train_X[train_columns].iloc[trn_idx], train_X[train_columns].iloc[val_idx]
    y_tr, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]

    model = CatBoostRegressor(n_estimators=25000, verbose=-1, objective="MAE", loss_function="MAE", boosting_type="Ordered", task_type="GPU")
    model.fit(X_tr, 
              y_tr, 
              eval_set=[(X_val, y_val)], 
#               eval_metric='mae',
              verbose=2500, 
              early_stopping_rounds=500)
    oof[val_idx] = model.predict(X_val)

    #feature importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = train_columns
    fold_importance_df["importance"] = model.feature_importances_[:len(train_columns)]
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    #predictions
#     if PREDICTION:

    predictions += model.predict(test_X[train_columns]) / folds.n_splits
    train_score.append(model.best_score_['learn']["MAE"])

cv_score = mean_absolute_error(train_y, oof)
print(f"After {n_fold} test_CV = {cv_score:.3f} | train_CV = {np.mean(train_score):.3f} | {cv_score-np.mean(train_score):.3f}", end=" ")

fold 0
0:	learn: 5.6534245	test: 5.7103149	best: 5.7103149 (0)	total: 73.6ms	remaining: 30m 40s
2500:	learn: 1.6521669	test: 1.7298248	best: 1.7298248 (2500)	total: 2m 34s	remaining: 23m 9s
5000:	learn: 1.4267563	test: 1.5685912	best: 1.5685912 (5000)	total: 5m 11s	remaining: 20m 46s
7500:	learn: 1.2687459	test: 1.4671061	best: 1.4671061 (7500)	total: 7m 49s	remaining: 18m 15s
10000:	learn: 1.1513249	test: 1.4000789	best: 1.4000789 (10000)	total: 10m 28s	remaining: 15m 42s
12500:	learn: 1.0538957	test: 1.3480049	best: 1.3480049 (12500)	total: 13m 9s	remaining: 13m 9s
15000:	learn: 0.9742160	test: 1.3100121	best: 1.3100121 (15000)	total: 15m 50s	remaining: 10m 33s
17500:	learn: 0.9065089	test: 1.2804751	best: 1.2804751 (17500)	total: 18m 30s	remaining: 7m 55s
20000:	learn: 0.8466090	test: 1.2553224	best: 1.2553155 (19998)	total: 21m 12s	remaining: 5m 17s
22500:	learn: 0.7949645	test: 1.2349034	best: 1.2349034 (22500)	total: 23m 53s	remaining: 2m 39s
24999:	learn: 0.7497545	test: 1.21891

In [11]:
today = str(datetime.date.today())
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')

submission["time_to_failure"] = predictions
submission.to_csv(f'CatBoost_{today}_test_{cv_score:.3f}_train_{np.mean(train_score):.3f}.csv', index=False)
submission.head()

,seg_id,time_to_failure
0,seg_00030f,2.816328
1,seg_0012b5,5.164259
2,seg_00184e,4.861732
3,seg_003339,8.675453
4,seg_0042cc,6.719634


<pre>Keras Neural Network</pre>

In [12]:
def create_model(input_dim=10):
    model = Sequential()
    model.add(Dense(256, activation="relu",input_dim=input_dim))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation="relu", bias_regularizer= l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="linear"))
 
    opt = adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #'rmsprop'
    # Compile model
    model.compile(
        loss='mae',
        optimizer=opt,
    )
    return model

patience = 50
call_ES = EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=1, mode='auto', baseline=None, restore_best_weights=True)

In [13]:
%%time
# n_fold = 5
folds = KFold(n_splits=n_fold, shuffle = True, random_state=42)

NN_oof = np.zeros(len(train_X))
train_score = []
fold_idxs = []

NN_predictions = np.zeros(len(test_X))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X,train_y.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    fold_idxs.append(val_idx)
    
    X_tr, X_val = train_X[train_columns].iloc[trn_idx], train_X[train_columns].iloc[val_idx]
    y_tr, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]
    model = create_model(train_X.shape[-1])
    model.fit(X_tr, y_tr, epochs=500, batch_size=32, verbose=0, callbacks=[call_ES,], validation_data=[X_val, y_val]) #
    
    NN_oof[val_idx] = model.predict(X_val)[:,0]
    
    NN_predictions += model.predict(test_X[train_columns])[:,0] / folds.n_splits
    history = model.history.history
    tr_loss = history["loss"]
    val_loss = history["val_loss"]
    print(f"loss: {tr_loss[-patience]:.3f} | val_loss: {val_loss[-patience]:.3f} | diff: {val_loss[-patience]-tr_loss[-patience]:.3f}")
    train_score.append(tr_loss[-patience])
#     break
    
cv_score = mean_absolute_error(train_y, NN_oof)
print(f"After {n_fold} test_CV = {cv_score:.3f} | train_CV = {np.mean(train_score):.3f} | {cv_score-np.mean(train_score):.3f}", end=" ")

fold 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Restoring model weights from the end of the best epoch
Epoch 00294: early stopping
loss: 1.309 | val_loss: 1.768 | diff: 0.458
fold 1
Restoring model weights from the end of the best epoch
Epoch 00143: early stopping
loss: 1.719 | val_loss: 1.892 | diff: 0.174
fold 2
Restoring model weights from the end of the best epoch
Epoch 00217: early stopping
loss: 1.392 | val_loss: 1.776 | diff: 0.383
fold 3
Restoring model weights from the end of the best epoch
Epoch 00209: early stopping
loss: 1.403 | val_loss: 1.649 | diff: 0.245
fold 4
Restoring model weights from the end of the best epoch
Epoch 00141: early stopping
loss: 1.712 | val_loss: 1.905 | diff: 0.193
After 5 test_CV = 1.692 | train_CV = 1.507 | 0.185 CPU times: user 1h 11min 11s, sys: 7min 22s,

In [14]:
today = str(datetime.date.today())
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')

submission["time_to_failure"] = NN_predictions
submission.to_csv(f'NN_{today}_test_{cv_score:.3f}_train_{np.mean(train_score):.3f}.csv', index=False)
submission.head()

,seg_id,time_to_failure
0,seg_00030f,3.001206
1,seg_0012b5,4.764928
2,seg_00184e,4.231320
3,seg_003339,7.579152
4,seg_0042cc,5.665317


# Final Submission

In [15]:
Scirpus_prediction = pd.read_csv("../input/andrews-new-script-plus-a-genetic-program-model/gpI.csv")
Scirpus_prediction.head()

,seg_id,time_to_failure
0,seg_00030f,2.856751
1,seg_0012b5,4.189392
2,seg_00184e,5.538267
3,seg_003339,9.051419
4,seg_0042cc,7.099581


In [16]:
today = str(datetime.date.today())
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')

submission["time_to_failure"] = (predictions+NN_predictions+Scirpus_prediction.time_to_failure.values)/3
submission.to_csv(f'FINAL_{today}_submission.csv', index=False)
submission.head()

,seg_id,time_to_failure
0,seg_00030f,2.891428
1,seg_0012b5,4.706193
2,seg_00184e,4.877106
3,seg_003339,8.435341
4,seg_0042cc,6.494844
